In [3]:
# from gensim.models import Word2Vec
# from itertools import *
# dimen = 128
# window = 5

In [1]:
import string;
import re;
import random
import math
import numpy as np
import torch
import os
from collections import defaultdict, Counter
from tqdm import tqdm
from torch_geometric.data import HeteroData

In [2]:
data = torch.load("data/data.pt")
print(data)

HeteroData(
  m={
    num_nodes=4666,
    y=[4666]
  },
  a={ num_nodes=5845 },
  d={ num_nodes=2271 },
  m_text_embed={ x=[4666, 128] },
  m_net_embed={ x=[4666, 128] },
  m_a_net_embed={ x=[4666, 128] },
  m_d_net_embed={ x=[4666, 128] },
  a_net_embed={ x=[5845, 128] },
  a_text_embed={ x=[5845, 128] },
  d_net_embed={ x=[2271, 128] },
  d_text_embed={ x=[2271, 128] },
  (m, walk, m)={ edge_index=[2, 32862] },
  (m, walk, a)={ edge_index=[2, 29894] },
  (m, walk, d)={ edge_index=[2, 13656] },
  (a, walk, m)={ edge_index=[2, 26087] },
  (a, walk, a)={ edge_index=[2, 35045] },
  (a, walk, d)={ edge_index=[2, 15898] },
  (d, walk, m)={ edge_index=[2, 11136] },
  (d, walk, a)={ edge_index=[2, 14009] },
  (d, walk, d)={ edge_index=[2, 6256] }
)


In [3]:
data_path = "data/"

In [17]:
# m_a_dict = defaultdict(list)

# with open(data_path + "m_a_list.txt", 'r') as file:            
#     lines = file.readlines()

#     for i, line in enumerate(lines):
#         enteries = re.split(',', line)
#         m_node, a_node = enteries[0].strip(), enteries[1].strip()
        
#         m_a_dict[f'm{m_node}'].append(f'a{a_node}')

In [18]:
# m_d_dict = defaultdict(list)

# with open(data_path + "m_d_list.txt", 'r') as file:            
#     lines = file.readlines()

#     for i, line in enumerate(lines):
#         enteries = re.split(',', line)
#         m_node, d_node = enteries[0].strip(), enteries[1].strip()
        
#         m_d_dict[f'm{m_node}'].append(f'd{d_node}')

In [19]:
# print(m_a_dict["m456"])
# print(m_d_dict["m4"])

['a291', 'a1783', 'a1597']
['d1763']


In [22]:
# with open(data_path + "metapath_list.txt", 'w') as file:  

#     for m_node in list(m_d_dict.keys()):
#         d_node = m_d_dict[m_node][0]
#         for a_node in m_a_dict[m_node]:
#             file.write(f'{m_node} {d_node} {a_node}\n')

In [23]:
# with open(data_path + "metapath_list2.txt", 'w') as file:  

#     for m_node in list(m_d_dict.keys()):
#         d_node = m_d_dict[m_node][0]
#         a_node = ' '.join(m_a_dict[m_node])
#         file.write(f'{m_node} {d_node} {a_node}\n')

In [4]:
with open(data_path + "metapath_list.txt", 'r') as file:            
    lines = file.readlines()

mp_num = 0
m_embed = (data['m_text_embed'].x + data['m_net_embed'].x) / 2
a_embed = (data['a_text_embed'].x + data['a_net_embed'].x) / 2
d_embed = (data['d_text_embed'].x + data['d_net_embed'].x) / 2

main_node = 0
embed = None
embedding = None
print(m_embed.shape)

with open(data_path + "mp1_embed.txt", 'w') as file1: 
    
    with open(data_path + "mp2_embed.txt", 'w') as file2: 
        
        with open(data_path + "mp3_embed.txt", 'w') as file3: 
            
            for i, line in enumerate(lines):
                
                enteries = re.split(' ', line.strip())
                m_node, d_node, a_node = int(enteries[0].strip()[1:]), int(enteries[1].strip()[1:]), int(enteries[2].strip()[1:])
                
                if main_node == m_node and i > 0: mp_num += 1
                elif main_node != m_node and mp_num < 2: 
                    if mp_num == 1:
                        file3.write(f'\n')
                    else:
                        file2.write(f'\n')
                        file3.write(f'\n')
                    mp_num = 0
                else: mp_num = 0
                
                if mp_num == 0: output_file = file1
                elif mp_num == 1: output_file = file2
                else: output_file = file3
                    
                embed = torch.cat((m_embed[m_node, :], a_embed[a_node, :], d_embed[d_node, :]))
                embedding = ' '.join(map(str, embed.tolist()))

                output_file.write(f'{embedding}\n')
                main_node = m_node

torch.Size([4666, 128])


In [73]:
# mp_tensor = torch.empty(0)
# mp_list = []
# a=-1

# with open(data_path + "metapath_list2.txt", 'r') as file:
#     lines = file.readlines()

#     for i, line in enumerate(lines):
#         entries = re.split(' ', line.strip())
#         node_list = [int(entry[1:]) for entry in entries]
#         if len(node_list) == 4: 
#             node_list.append(a)
#             a-= 1
#         if len(node_list) == 3: 
#             node_list.append(a)
#             a-=1
#             node_list.append(a)
#             a-=1
#         # print(node_list)
#         mp_list.append(torch.tensor([node_list]))
        
#     mp_tensor =  torch.cat(mp_list, dim = 0)

In [74]:
# print(mp_tensor)

tensor([[   0,  577, 3760, 4074,   87],
        [   1, 1272, 5352, 1331, 2324],
        [   2, 1074, 1509, 1988, 2916],
        ...,
        [4663,  216, 4970, 4927, 3447],
        [4664, 1309, 5249, 1192,  161],
        [4665,  775, 4236, 3194,  999]])


In [115]:
# def find_matching_pairs(column_tensor):
#     num_rows = column_tensor.size(0)
#     matching_pairs = []

#     for i in tqdm(range(num_rows), desc="Processing rows"):
#         for j in range(i + 1, num_rows):

#             source = [column_tensor[i].item()] if column_tensor[i].numel() == 1 else column_tensor[i,:].tolist()
#             target = [column_tensor[j].item()] if column_tensor[j].numel() == 1 else column_tensor[j,:].tolist()
#             for k in source: 
#                 if k in target:
#                     if (i,j) == (1,12): print("poop")
#                     matching_pairs.append((i, j))

#     return matching_pairs

In [87]:
# print(mp_tensor[:,1])
# from tqdm import tqdm

tensor([ 577, 1272, 1074,  ...,  216, 1309,  775])


In [117]:
# pairs1 = find_matching_pairs(mp_tensor[:,1])

Processing rows:   0%|          | 8/4666 [00:00<02:16, 34.14it/s]

poop


Processing rows: 100%|██████████| 4666/4666 [01:33<00:00, 50.09it/s] 


In [116]:
# pairs2 = find_matching_pairs(mp_tensor[:,2:])

Processing rows:   0%|          | 6/4666 [00:00<03:08, 24.79it/s]

poop
poop
poop


Processing rows: 100%|██████████| 4666/4666 [02:03<00:00, 37.91it/s] 


In [118]:
# pairs = pairs1+pairs2
# print(len(pairs1), len(pairs2), len(pairs))

7509 43987 51496


In [110]:
# from collections import Counter

# def write_unique_tuples_to_file(pairs, file_path):
#     # Count the occurrences of each tuple
#     tuple_counts = Counter(pairs)

#     with open(file_path, 'w') as file:
#         for pair, count in tuple_counts.items():
#             # Write the tuple along with its count
#             file.write(f"{pair[0]} {pair[1]} {count}\n")

In [111]:
# write_unique_tuples_to_file(pairs, "data/edges_list.txt")

In [5]:
M_n = 4666
embed_d = 128*3
data_path = "data/"

In [6]:
mp1_embed = torch.zeros(M_n, embed_d, dtype=torch.float32)
mp2_embed = torch.zeros(M_n, embed_d, dtype=torch.float32)
mp3_embed = torch.zeros(M_n, embed_d, dtype=torch.float32)

f_names = ["mp1_embed.txt", "mp2_embed.txt", "mp3_embed.txt"]

for f_name in f_names:
    with open(data_path + f_name, 'r') as file:            
        lines = file.readlines()      
			
        for i, line in enumerate(lines):
            entries = line.strip().split()
            if entries == []: continue
            if f_name == 'mp1_embed.txt':
                mp1_embed[i] = torch.tensor([float(x) for x in entries]) 
            elif f_name == 'mp2_embed.txt':
                mp2_embed[i] = torch.tensor([float(x) for x in entries]) 
            else:
                mp3_embed[i] = torch.tensor([float(x) for x in entries]) 

In [7]:
m_class = torch.full((M_n,), -1, dtype=torch.long)
with open(data_path + "m_class.txt", 'r') as file:            
    lines = file.readlines()
    for i, line in enumerate(lines):
        entries =  line.strip().split(',')
        m_class[int(entries[0])] = int(entries[1].strip())

In [8]:
print(m_class.shape, mp1_embed.shape, mp2_embed.shape)

torch.Size([4666]) torch.Size([4666, 384]) torch.Size([4666, 384])


In [9]:
m_a_edge_index = torch.empty(0)
m_a_edge_weight = torch.zeros((50253,), dtype=torch.float32)
m_a_list = []

with open(data_path + "edges_list.txt", 'r') as file:
    lines = file.readlines()
    
    for i, line in enumerate(lines):
        entries =  line.strip().split()
        edge_list = [int(x) for x in entries[0:2]]
        weight = float(entries[2])
        if i < 5: print(edge_list, weight)
        m_a_list.append(torch.tensor([edge_list]))
        m_a_edge_weight[i] = weight

    # Concatenate the list of tensors into a single tensor
    m_a_edge_index = torch.cat(m_a_list, dim=0).t().contiguous()

[0, 25] 1.0
[0, 279] 1.0
[0, 282] 1.0
[0, 585] 1.0
[0, 2392] 1.0


In [10]:
print(m_a_edge_index.shape, m_a_edge_weight.shape)

torch.Size([2, 50253]) torch.Size([50253])


In [11]:
data = HeteroData()
    
data['m'].num_nodes = 4666

data['mp1'].x = mp1_embed
data['mp2'].x = mp2_embed
data['mp3'].x = mp3_embed
    
data['mp'].edge_index = m_a_edge_index
data['mp'].edge_weight = m_a_edge_weight 
data['mp'].y = m_class

In [12]:
print(data)

HeteroData(
  m={ num_nodes=4666 },
  mp1={ x=[4666, 384] },
  mp2={ x=[4666, 384] },
  mp3={ x=[4666, 384] },
  mp={
    edge_index=[2, 50253],
    edge_weight=[50253],
    y=[4666]
  }
)


In [37]:
torch.save(data, 'data.pt')